<a href="https://colab.research.google.com/github/victor-wildlife/wild_about_ai_classifications/blob/main/wild_about_ai_classifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title <font size="5">↓ ឵឵<i>Download latest zooniverse classifications</font> { vertical-output: true }

import gdown
url = "https://drive.google.com/drive/folders/12GAX8TZegfS1_Q8bbp4IZEJJ_Mj_FVG0"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/zooniverse_classifications/spyfish-aotearoa-classifications.csv',
 '/content/zooniverse_classifications/spyfish-aotearoa-subjects.csv',
 '/content/zooniverse_classifications/spyfish-aotearoa-workflows.csv']

In [8]:
# @title <font size="5">↓ ឵឵<i>Read zooniverse classifications as df</font> { vertical-output: true }

import pandas as pd
import os 

directory = '/content/zooniverse_classifications'

for file in os.listdir(directory):
  
  file_path = os.path.join(directory, file)
  
  if "workflows" in file:
    workflows_df = pd.read_csv(file_path)
  
  if "classifications" in file:
    classifications_df = pd.read_csv(file_path)
  
  if "subjects" in file:
    subjects_df = pd.read_csv(file_path)


In [25]:
# @title <font size="5">↓ ឵឵<i>Filter classifications by username</font> { vertical-output: true }
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a text input widget for the username
username_widget = widgets.Text(
    value='',
    placeholder='Enter your Zooniverse username',
    description='Zooniverse username:',
    disabled=False
)
# Define a function to filter the dataframe by username
def filter_dataframe(username):
    if username in classifications_df.user_name.unique():
      filtered_classifications = classifications_df[classifications_df['user_name'] == username]
      print("Selected", filtered_classifications.shape[0], "classifications from", username)

      return filtered_classifications
  
    else:
      raise ValueError(
                "The username doesn't exist or doesn't have classifications any subjects in this project"
            )

# Create a button to filter the dataframe
filter_button = widgets.Button(description='Get classifications')

def filter_classifications(button):
    with out:
        clear_output()
        # Select classifications provided by the username of interest
        filtered_df = filter_dataframe(username_widget.value)
        
        # Flatten the classifications
        
        
        # Download the classifications
        filtered_df.to_csv('classifications_zoo.csv', index=False)


# Display the widgets
display(username_widget)
display(filter_button)

out = widgets.Output()
display(out)


filtered_classifications = filter_button.on_click(filter_classifications)

Text(value='', description='Zooniverse username:', placeholder='Enter your Zooniverse username')

Button(description='Get classifications', style=ButtonStyle())

Output()

In [27]:
# from ipywidgets import Output, Button
# from IPython.display import HTML, clear_output
# from base64 import b64encode

# download_output = Output()
# display(download_output)
# def trigger_download(text, filename, kind='text/json'):
#     # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
#     content_b64 = b64encode(text.encode()).decode()
#     data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
#     js_code = f"""
#         var a = document.createElement('a');
#         a.setAttribute('download', '{filename}');
#         a.setAttribute('href', '{data_url}');
#         a.click()
#     """
#     with download_output:
#         clear_output()
#         display(HTML(f'<script>{js_code}</script>'))

# btn = Button(description='download some shit')
# def download_shit(e=None):
#     trigger_download('shit', 'shit.txt', kind='text/plain')
# btn.on_click(download_shit)
# display(btn)

Output()

Button(description='download some shit', style=ButtonStyle())

In [ ]:
# # Combine annotation and subject information
#     annotations_df = pd.merge(
#         annotations,
#         subjects_df,
#         how="left",
#         left_on="subject_ids",
#         right_on="id",
#         validate="many_to_one",
#     )


In [ ]:
# Extract metadata from uploaded subjects
#     subjects_df, subjects_meta = extract_metadata(subjects)

#     # Combine metadata info with the subjects df
#     subjects = pd.concat([subjects_df, subjects_meta], axis=1)

#     # Check if the Zooniverse project is the Spyfish
#     if project_name == "Spyfish_Aotearoa":
#         subjects = process_spyfish_subjects(subjects, db_path)

#     # If project is not KSO or Spyfish standardise subject info
#     else:
#         # Create columns to match schema if they don't exist
#         subjects["frame_exp_sp_id"] = subjects.get("frame_exp_sp_id", np.nan)
#         subjects["frame_number"] = subjects.get("frame_number", np.nan)

#         # Select only relevant metadata columns
#         subjects = subjects[
#             [
#                 "subject_id",
#                 "project_id",
#                 "workflow_id",
#                 "subject_set_id",
#                 "locations",
#                 "movie_id",
#                 "frame_number",
#                 "frame_exp_sp_id",
#                 "upl_seconds",
#                 "Subject_type",
#                 "subject_type",
#                 "#VideoFilename",
#                 "#clip_length",
#                 "classifications_count",
#                 "retired_at",
#                 "retirement_reason",
#                 "created_at",
#             ]
#         ]

#         # Fix weird bug where Subject_type is used instead of subject_type for the column name for some clips
#         if "Subject_type" in subjects.columns:
#             subjects["subject_type"] = subjects[
#                 ["subject_type", "Subject_type"]
#             ].apply(lambda x: x[1] if isinstance(x[1], str) else x[0], 1)
#             subjects.drop(columns=["Subject_type"], inplace=True)

#         # Rename columns to match the db format
#         subjects = subjects.rename(
#             columns={
#                 "#VideoFilename": "filename",
#                 "upl_seconds": "clip_start_time",
#                 "#frame_number": "frame_number",
#             }
#         )

#         # Remove clip subjects with no clip_start_time info (from different projects)
#         subjects = subjects[
#             ~(
#                 (subjects["subject_type"] == "clip")
#                 & (subjects["clip_start_time"].isna())
#             )
#         ]

#         # Calculate the clip_end_time
#         subjects["clip_end_time"] = (
#             subjects["clip_start_time"] + subjects["#clip_length"]
#         )

# # Set subject_id information as id
# subjects = subjects.rename(columns={"subject_id": "id"})

# # Extract the html location of the subjects
# subjects["https_location"] = subjects["locations"].apply(
#     lambda x: literal_eval(x)["0"]
# )

# # Set movie_id column to None if no movies are linked to the subject
# if movie_folder == "None" and server in ["LOCAL", "SNIC"]:
#     subjects["movie_id"] = None

# # Fix subjects where clip_start_time is not provided but upl_seconds is
# if "clip_start_time" in subjects.columns and "upl_seconds" in subjects.columns:
#     subjects["clip_start_time"] = subjects[
#         ["clip_start_time", "upl_seconds"]
#     ].apply(lambda x: x[0] if not np.isnan(x[0]) else x[1], 1)

# # Set the columns in the right order
# subjects = subjects[
#     [
#         "id",
#         "subject_type",
#         "filename",
#         "clip_start_time",
#         "clip_end_time",
#         "frame_exp_sp_id",
#         "frame_number",
#         "workflow_id",
#         "subject_set_id",
#         "classifications_count",
#         "retired_at",
#         "retirement_reason",
#         "created_at",
#         "https_location",
#         "movie_id",
#     ]
# ]